## BERT-wwm 汉语言模型
名为 BERT-Whole Word Masking，是 BERT 模型的改进版本。

它是使用全词掩码方法进行预训练的。概括地说，在全词掩码中，如果子词被掩盖了，那么就**掩盖了与该子词对应的所有单词**

In [1]:
! pip install transformers==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 17.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
from transformers import BertTokenizer, BertModel

In [3]:
# 加载哈工大 BERT-wwm-chinese 的分词器和预训练模型
tokenizer = BertTokenizer.from_pretrained('hfl/chinese-bert-wwm')
model = BertModel.from_pretrained('hfl/chinese-bert-wwm')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [4]:
model.config

BertConfig {
  "_name_or_path": "hfl/chinese-bert-wwm",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

对文本进行分词

In [5]:
# 输入一个中文句子
sentence = "哈尔滨工业大学是中国著名的理工类高校。"

# 使用 BERT-wwm 的 tokenizer 进行分词
tokens = tokenizer.tokenize(sentence)
print("分词结果:", tokens)

# 将分词结果转为模型输入所需的ID
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
print(inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


分词结果: ['哈', '尔', '滨', '工', '业', '大', '学', '是', '中', '国', '著', '名', '的', '理', '工', '类', '高', '校', '。']
{'input_ids': tensor([[ 101, 1506, 2209, 4012, 2339,  689, 1920, 2110, 3221,  704, 1744, 5865,
         1399, 4638, 4415, 2339, 5102, 7770, 3413,  511,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


特征提取

In [6]:
# 使用模型获取输出
outputs = model(**inputs)
# 获取最后一层隐藏状态
last_hidden_states = outputs.last_hidden_state

print("模型输出的 hidden states:", last_hidden_states)

模型输出的 hidden states: tensor([[[-0.5444,  0.6245,  0.3091,  ...,  0.9211,  0.5568, -0.1091],
         [ 0.7777, -0.8819, -0.1788,  ...,  1.1876,  0.3116, -0.3767],
         [ 0.5350, -0.9537, -0.1746,  ...,  1.5746,  0.3200,  0.0569],
         ...,
         [ 0.6980,  0.3168,  0.3564,  ...,  1.4848,  1.1309,  0.0796],
         [ 0.0436,  0.2924, -0.0586,  ...,  0.4487,  0.3403, -0.0255],
         [-0.3210,  0.7911, -0.3371,  ...,  1.6507,  0.4039,  0.1568]]],
       grad_fn=<NativeLayerNormBackward0>)


In [7]:
print(last_hidden_states.shape)

torch.Size([1, 21, 768])


应用场景：文本分类

In [8]:
from transformers import BertForSequenceClassification

`num_labels=2`: 表示分类任务中有 2 个类别（用于二分类）

In [9]:
# 加载哈工大的 BERT-wwm-chinese 的分类模型
classifier_model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`padding=True`: 自动对较短的输入进行填充，使其长度与 batch 中的最长句子保持一致。

`truncation=True`: 自动对过长的输入进行截断，确保输入长度不会超过模型的最大限制

In [20]:
sentence = "我会喜欢你一辈子的！"

# 输入文本进行编码
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
print(inputs)

{'input_ids': tensor([[ 101, 2769,  833, 1599, 3614,  872,  671, 6777, 2094, 4638, 8013,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [21]:
# 通过分类器模型进行情感预测
outputs = classifier_model(**inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0087,  0.1513]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [22]:
logits = outputs.logits
print(logits)

tensor([[-0.0087,  0.1513]], grad_fn=<AddmmBackward0>)


In [23]:
# 获取预测的类别
predicted_class = logits.argmax(dim=1).item()

In [24]:
if predicted_class == 0:
    print("负面情感")
else:
    print("正面情感")

正面情感
